Before running this code a number of steps need to be performed. Most of it can be found here:
https://python.gotrained.com/youtube-api-extracting-comments/

# Installing Google API client library

In [1]:
!pip install google-api-python-client
!pip install google-auth google-auth-oauthlib google-auth-httplib2

     |████████████████████████████████| 142 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 76 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 582 kB 25.2 MB/s eta 0:00:01
  Created wheel for google-api-python-client: filename=google_api_python_client-1.7.11-py3-none-any.whl size=55852 sha256=ddc89bf3c4e9bd2f6e3835d0ff5ed54141321bf6868281f54ff409fa121b9b2b
  Stored in directory: /Users/maggieilieva/Library/Caches/pip/wheels/a2/02/a4/21d46730d22a702c4927fbc9fb34b435e37c80f8303b21a6fd
Successfully built google-api-python-client
ERROR: tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 46.0.0 which is incompatible.
  Attempting uninstall: setuptools
    Found existing installation: setuptools 40.2.0
    Uninstalling setuptools-40.2.0:
      Successfully uninstalled setuptools-40.2.0
     |████████████████████████████████| 147 kB 6.0 MB/s eta 0:00:01


In [2]:
!pip install pandas

# Setting up Google API client

In [44]:
CLIENT_SECRETS_FILE = "client_secret.json" #This file has to be in the same folder as the ipynb file.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

import os
import pickle
import google.oauth2.credentials
import dateutil.parser
import datetime
import pytz

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


In [12]:
service = get_authenticated_service()

In [115]:
def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()
    
    utc=pytz.UTC
    
    ####### ENTER RELEASE DATE HERE #######
    release_date = utc.localize(datetime.datetime(2020, 3, 3)) #year, month, day

    while results:
        for item in results['items']:
            date = item['snippet']['topLevelComment']['snippet']['publishedAt']
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            parsed_date = dateutil.parser.parse(date)
            if parsed_date < release_date:
                comments.append(comment)

        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments

In [14]:
import csv

def write_to_csv(comments):
    with open('comments.csv', 'w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['Video ID', 'Comment'])
        for row in comments:
            comments_writer.writerow(list(row))

Get the comments and put them into a file 'comments.csv' that is created in your ipynb file directory.

In [122]:
def search_videos_by_keyword(service, **kwargs):
    final_result = []
    
    video_id = '4KlFmkbU' #insert movie ID here
    
    comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
    final_result.extend([(video_id, comment) for comment in comments])
    write_to_csv(final_result)


search_videos_by_keyword(service, q='', part='id,snippet', eventType='completed', type='video')


HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=4KlFmkbU&textFormat=plainText&alt=json returned "Daily Limit Exceeded. The quota will be reset at midnight Pacific Time (PT). You may monitor your quota usage and adjust limits in the API Console: https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=21425087277">

In [112]:
with open('comments.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row,'\n')

['Video ID', 'Comment'] 

['5xH0HfJHsaY', 'Movies like Parasite, Roma, and Shoplifters are the reason why we should respect non-english films.'] 

['5xH0HfJHsaY', "I thought the movie was something like horror, terror, brutal etc. because of the title itself😂😅 but after watching the whole movie, tbh I'm crying... The lesson of the movie is deep.. Love it. Recommend it to someone without giving any context..just tell them the movie is perfect.\n(guide your children when watching Parasite, there is a part that really unexpected😂😂, you know what I mean to someone who already watched the movie😂)"] 

['5xH0HfJHsaY', 'One of the best directed movies ive seen with superb attention to detail. Everything ties in at the end. Every scene is purposeful. Felt numb at the cinema after i walked out with the level of suspense'] 

['5xH0HfJHsaY', 'Masterpiece.'] 

['5xH0HfJHsaY', 'the first thing come in my mind is what the title means Parasite i wonder that its look like creepy things and i  cannot ge

['5xH0HfJHsaY', 'So what is about \nWhether rich people are nice or not??? 🧐🧐'] 

['5xH0HfJHsaY', "Just watched  still can't get over \n\n\n\nExpectancy to b unexpectancy 😯"] 

['5xH0HfJHsaY', 'Best movie.. become a fan from south india, india'] 

['5xH0HfJHsaY', 'But still you guys will eat insects and bats 🤣'] 

['5xH0HfJHsaY', "Not gonna lie I thought it was Japanese at first then Chinese, but now I now it's korean"] 

['5xH0HfJHsaY', 'its a masterpiece by Bong Joon Ho. A must watch movie......!  Its a story of 2 families Rich & Poor......! After watching this movie U would be in dilemma that who is right and who is wrong.......!'] 

['5xH0HfJHsaY', 'മലയാളീസ് പ്ലീസ് കമോൺ...'] 

['5xH0HfJHsaY', 'Great film but thought 1917 was better.'] 

['5xH0HfJHsaY', 'I am glad this one got the recognition it deserves'] 

['5xH0HfJHsaY', 'Just finished watching it!!!! Brilliant!!!'] 

['5xH0HfJHsaY', 'Is this the Anime?'] 

['5xH0HfJHsaY', 'Hollywood crying no diversity come award time.\n\nParasi

['5xH0HfJHsaY', 'Wtf is this'] 

['5xH0HfJHsaY', 'Congratulations from a fan of Tamil cinema .... appreciating the academy awards for being diverse ....'] 

['5xH0HfJHsaY', 'Who is here after *4 OSCAR AWARDS FOR PARASITE*'] 

['5xH0HfJHsaY', "I heard alot about this movie before the oscars now I'm really curious to know what is it about because obviously the trailer isn't showing much of it but it still seems very interesting"] 

['5xH0HfJHsaY', 'i mean i loved the anime, ill check this out'] 

['5xH0HfJHsaY', "I don't know where to Focus..\n Video or Substitles.😁"] 

['5xH0HfJHsaY', 'Am I the only one who laughed when she said "deadry serious"?'] 

['5xH0HfJHsaY', 'This one at the oscars and it wasn’t even in English   Wtf'] 

['5xH0HfJHsaY', "I'm so happy it won Oscars,it got what it fuckin' deserved."] 

['5xH0HfJHsaY', 'Who is here after Oscar?'] 

['5xH0HfJHsaY', 'How many peoples  after the Oscar 👀'] 

['5xH0HfJHsaY', 'Parasite won the Oscar\nLe me : ab to dekhna hi hoga\n       


['5xH0HfJHsaY', '... what?'] 

['5xH0HfJHsaY', 'Must watch this. Oscar brought me here ..'] 

['5xH0HfJHsaY', '"one of the best movies of the year" - nope, quite frankly.. the audience thought it actually was THE best movie of the year.'] 

['5xH0HfJHsaY', 'who is here from India after oscar?'] 

['5xH0HfJHsaY', 'Anyone else who has no idea why it won the oscar the best picture?'] 

['5xH0HfJHsaY', 'Who is here after Oscar..?'] 

['5xH0HfJHsaY', "Hey who's here after best picture❤"] 

['5xH0HfJHsaY', "Who's here after it won 4 Oscar award's.......?"] 

['5xH0HfJHsaY', 'Who is here after oscar award ..like here 👇👇'] 

['5xH0HfJHsaY', 'Congratulations for Oscar 👍💪'] 

['5xH0HfJHsaY', 'No one:\nEveryone:who’s here after the oscars'] 

['5xH0HfJHsaY', 'Absolute mastery.'] 

['5xH0HfJHsaY', 'I am not a fan of reading subtitles but I will defiantly gives this a try.'] 

['5xH0HfJHsaY', 'You will not understand anything by watching trailer just watch the movie'] 

['5xH0HfJHsaY', 'OSCAR KYA 


['5xH0HfJHsaY', 'Finally a damned brilliant movie after so much time. Hats off to the director, writers and the cast and also cinematographer.'] 

['5xH0HfJHsaY', 'who sat quietly 20  mins after the film ends?'] 

['5xH0HfJHsaY', 'ill pass'] 

['5xH0HfJHsaY', 'No clue about the movie'] 

['5xH0HfJHsaY', 'Yo pensé que era de terror🙁🙁🙁'] 

['5xH0HfJHsaY', "Just watched and I would be stunned if it doesn't win Best Picture."] 

['5xH0HfJHsaY', 'did anyone else think the characters in the poor family were extremely unlikeable? I’m poor myself but it was hard to sympathise with these characters because they did such horrible things. The film making aspect and technical aspects were absolutely incredible however. It’s just the message of the movie was kinda ruined because those characters were pretty bad people :/'] 

['5xH0HfJHsaY', 'Anna wintour brought me here'] 

['5xH0HfJHsaY', 'Coronna virus movie'] 

['5xH0HfJHsaY', "for everyone who hasn't watched Parasite, you aren't supposed to kn


['5xH0HfJHsaY', 'i just watched this film. Best decision'] 

['5xH0HfJHsaY', 'A LITERAL masterpiece. PERFECT. Deep af!!! It’s amazing!'] 

['5xH0HfJHsaY', 'Are the comments trolls? I watched and was not impressed. What is the hype about? I need someone to explain what made this movie "amazing".'] 

['5xH0HfJHsaY', 'This is how trailers supposed to be ...'] 

['5xH0HfJHsaY', "Where's the parasite"] 

['5xH0HfJHsaY', "I didn't get much from that but it's up for an Oscar so I guess it's a must watch.🤷\u200d♀️"] 

['5xH0HfJHsaY', 'I thought this was film about my ex wife'] 

['5xH0HfJHsaY', "I feel like those comments claiming it's the best movie so just go see it are part of ads"] 

['5xH0HfJHsaY', 'i thought it would be some alien movie not gold diggers damn'] 

['5xH0HfJHsaY', 'Is this a horror movie?'] 

['5xH0HfJHsaY', "where's migi?"] 

['5xH0HfJHsaY', 'It’s a must watch movie!'] 

['5xH0HfJHsaY', 'I here just for checking the comments.'] 

['5xH0HfJHsaY', '2 point of 10..'] 

['5xH

In [125]:
import pandas as pd

comment_df = pd.read_csv("comments.csv")

In [132]:
comment_df['Comment']

0                                                    Ez
1                  nooo, anything but not the baby foot
2     ❤️📖📖📖📖📖📖📕📕📕📕📕📕📕📕📐📐📐📐📐🖊🖊❤️❤️❤️🖤🖤🖤📴📴📴📴🆔📴🆔🆔🆔🆔🆔♏️♏...
3                 What's the song used in this trailer?
4     I think this film is my #1 guilty pleasure sin...
5     i don't wan't to like nor comment because it w...
6     "Conedy of the Summer"\nHAHAHA! REALLY? That's...
7                                                   NaN
8              the room: so bad its good\nthis: nothing
9                     Nostalgia Critic: Is Parody Dead?
10    If y'all think this movie is ''horrible'', the...
11    even a pile of crap makes laugh more than this...
12    that's the worst movie i have seen in my whole...
13                           This movie looks horrible.
14                                         Cool love it
15    I dont get what its about. I want to see if I ...
16                     its looks like a homemade movie.
17                     Well this movie is a Disa